In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv", parse_dates=['date'])
df.head()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",parse_dates=['date'])
test_df.head()

In [ ]:
submission = pd.read_csv(
    '/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv',
    index_col='row_id')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

## EDA

In [ ]:
sns.set()
plt.figure(figsize=(12,5))
g = sns.barplot(x='store', y = 'num_sold', data=df, ci=None)
g.bar_label(g.containers[0],fmt="%.0f", padding=2)
plt.show()

KaggleRama is outdoing the KaggleMart

In [ ]:
plt.figure(figsize=(12,5))
g = sns.barplot(x='country', y = 'num_sold', data=df, ci=None)
g.bar_label(g.containers[0],fmt="%.0f", padding=2)
plt.show()

Norway is having the highest no.s sold

In [ ]:
plt.figure(figsize=(12,5))
g=sns.barplot(x='product', y = 'num_sold', data=df, ci=None)
g.bar_label(g.containers[0],fmt="%.0f", padding=2)
plt.show()

Kaggle Hat is the best selling item

In [ ]:
plt.figure(figsize=(12,5))
g=sns.barplot(x='product', y = 'num_sold', hue='store', data=df, ci=None)
g.bar_label(g.containers[0],fmt="%.0f", padding=2)
g.bar_label(g.containers[1],fmt="%.0f", padding=2)
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
g = sns.barplot(x='country', y = 'num_sold', hue='store', data=df, ci=None)
g.bar_label(g.containers[0],fmt="%.0f", padding=2)
g.bar_label(g.containers[1],fmt="%.0f", padding=2)
plt.show()

from the above 3 plots we can see that KaggleRama is doing way better than KaggleMart.

## Using XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
y = df.pop('num_sold')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek


df = df.drop('date', axis='columns')

In [ ]:

test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['dayofweek'] = test_df['date'].dt.dayofweek

test_df = test_df.drop('date', axis='columns')

Encoding the data to fit it into the model

In [ ]:
country_map = {'Finland': 0, 'Sweden': 1, 'Norway': 2}
store_map = {'KaggleMart': 0, 'KaggleRama': 1}
product_map = {'Kaggle Mug': 0, 'Kaggle Hat': 1, 'Kaggle Sticker': 2}

df['country'] = df['country'].map(country_map)
df['store'] = df['store'].map(store_map)
df['product'] = df['product'].map(product_map)

In [ ]:
test_df['country'] = test_df['country'].map(country_map)
test_df['store'] = test_df['store'].map(store_map)
test_df['product'] = test_df['product'].map(product_map)

In [ ]:
xg_reg = xgb.XGBRegressor(objective="reg:linear", n_estimators=10, seed=123)
xg_reg.fit(df, y)
benchmark = submission.copy()
benchmark['num_sold'] = xg_reg.predict(test_df)
benchmark.to_csv('submission.csv')